In [ ]:
from ipywidgets import interact, interact_manual, widgets
import numpy as np
import matplotlib.pyplot as plt
import h5py
from copy import copy
from os import system

In [ ]:
system('wget patrickhaddad.xyz/data/traces/ex00.h5')

hdf5_file="ex00.h5"
root_name='carac'
data_file = h5py.File(hdf5_file,'r')
m_Y=data_file[root_name+ "/traces"][:512,150:]
m_key=data_file[root_name+ "/plaintexts"][:512,:]
m_cipher=data_file[root_name+ "/ciphertexts"][:512,:]
m_plain=data_file[root_name+ "/cipherkeys"][:512,:]
data_file.close()
nb_pts=m_Y.shape[1]
nb_traces=m_Y.shape[0]
v_Y_mean=m_Y.mean(0)
v_Y_std=m_Y.std(0)


In [ ]:
v_HW=np.zeros((256,),dtype=np.float64)
for i in range(256):
    v_HW[i]=bin(i).count('1')

def rotate(word, n):
    return word[n:]+word[0:n]

# Galois Multiplication
def galoisMult(a, b):
    p = 0
    hiBitSet = 0
    for i in range(8):
        if b & 1 == 1:
            p ^= a
        hiBitSet = a & 0x80
        a <<= 1
        if hiBitSet == 0x80:
            a ^= 0x1b
        b >>= 1
    return p % 256

v_rcon=np.array([0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a,0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39,0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a,0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8,0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef,0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc,0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b,0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3,0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94,0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20,0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35,0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f,0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04,0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63,0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd,0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb],dtype=np.uint8)
v_SBox=np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16 ],dtype=np.uint8)
v_shiftRows_map=np.zeros((16,),dtype=np.uint8)
state=np.arange(16,dtype=np.uint8)
for i in range(4):
    [v_shiftRows_map[i], v_shiftRows_map[i+4], v_shiftRows_map[i+8], v_shiftRows_map[i+12]] = rotate([state[i], state[i+4], state[i+8], state[i+12]], i)

m_galoisMult=np.zeros((3,256),dtype=np.uint8)
for i in range(3):
    for j in range(256):
        m_galoisMult[i,j]=galoisMult(j,i+1)
m_mc_table=np.ones((4,4),dtype=np.uint8)
for i in range(4):
    m_mc_table[i,i]=2
    m_mc_table[i,(i+1)%4]=3

def fast_key_schedule_128(m_key_i):
    N=4
    R=11
    nb_enc=m_key_i.shape[0]
    m_key_exp=np.zeros((nb_enc,R*16),dtype=np.uint8)
    for i in range(N*R):
        v_j=4*i+np.arange(4,dtype=np.uint8)
        if i<N:
            m_key_exp[:,v_j]=m_key_i[:,v_j]
        else:
            if i%N==0:
                rotw=m_key_exp[:,4*(i-1)+(np.arange(1,5,dtype=np.uint8)%4)]
                sbw=v_SBox[rotw]
                sbw[:,0]^=v_rcon[int(i/N)]
                m_key_exp[:,v_j]=m_key_exp[:,v_j-16]^sbw
            else:
                m_key_exp[:,v_j]=m_key_exp[:,v_j-16]^m_key_exp[:,v_j-4]
    return m_key_exp

def fast_aes_128(m_plain_in,m_key_exp_in):
    nb_enc=m_plain_in.shape[0]
    m_state_ret=np.zeros((21,nb_enc,16),dtype=np.uint8)
    m_state=copy(m_plain_in)
    cnt=0
    m_state_ret[cnt,:,:]=m_state
    cnt+=1
    for i in range(9):
        m_round_key=m_key_exp_in[:,16*i:16*(i+1)]
        m_state_sbsr=(v_SBox[m_state^m_round_key])[:,v_shiftRows_map]
        m_state_ret[cnt,:,:]=m_state_sbsr
        cnt+=1
        m_state_mix=np.zeros((nb_enc,16),dtype=np.uint8)
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    m_state_mix[:,4*j+k]^=m_galoisMult[m_mc_table[k,l]-1,m_state_sbsr[:,4*j+l]]
        m_state=copy(m_state_mix)
        m_state_ret[cnt,:,:]=m_state
        cnt+=1
    i=9
    m_round_key=m_key_exp_in[:,16*i:16*(i+1)]
    m_state_sbsr=(v_SBox[m_state^m_round_key])[:,v_shiftRows_map]
    m_state_ret[cnt,:,:]=m_state_sbsr
    cnt+=1
    i=10
    m_round_key=m_key_exp_in[:,16*i:16*(i+1)]
    m_cipher_tmp=m_state_sbsr^m_round_key
    m_state_ret[cnt,:,:]=m_cipher_tmp
    cnt+=1
    return m_state_ret


m_key_exp=fast_key_schedule_128(m_key)
m_state_t=fast_aes_128(m_plain,m_key_exp)

def plot_pearson_aes(state_name={'plain','SubByte0','MixCol0','SubByte1','MixCol1','SubByte2','MixCol2','SubByte3','MixCol3','SubByte4','MixCol4','SubByte5','MixCol5','SubByte6','MixCol6','SubByte7','MixCol7','SubByte8','MixCol8','SubByte9','AddKey9'},byte_to_attack=range(16)):
    if state_name=='plain':
        state_index=0
    elif state_name=='SubByte0':
        state_index=1
    elif state_name=='MixCol0':
        state_index=2
    elif state_name=='SubByte1':
        state_index=3
    elif state_name=='MixCol1':
        state_index=4
    elif state_name=='SubByte2':
        state_index=5
    elif state_name=='MixCol2':
        state_index=6
    elif state_name=='SubByte3':
        state_index=7
    elif state_name=='MixCol3':
        state_index=8
    elif state_name=='SubByte4':
        state_index=9
    elif state_name=='MixCol4':
        state_index=10
    elif state_name=='SubByte5':
        state_index=11
    elif state_name=='MixCol5':
        state_index=12
    elif state_name=='SubByte6':
        state_index=13
    elif state_name=='MixCol6':
        state_index=14
    elif state_name=='SubByte7':
        state_index=15
    elif state_name=='MixCol7':
        state_index=16
    elif state_name=='SubByte8':
        state_index=17
    elif state_name=='MixCol8':
        state_index=18
    elif state_name=='SubByte9':
        state_index=19
    elif state_name=='AddKey9':
        state_index=20

    else:    
        state_index=0
    v_x=v_HW[m_state_t[state_index,:,byte_to_attack]]
    v_r2=abs(((np.transpose(np.broadcast_to(v_x,(nb_pts,nb_traces)))*m_Y).mean(0)-v_Y_mean*(v_x.mean()))/(v_Y_std*(v_x.std())))
    max_index=np.argmax(v_r2)
    max_value=v_r2[max_index]
    
    plt.figure(figsize=[10,6])
    plt.plot(v_r2)
    plt.plot([max_index],[max_value],'+')
    plt.text(max_index+5, max_value+0.01, 'MAXIMUM=%.2f a l instant %d'%(max_value,max_index), fontsize=12,color='tab:blue')

In [9]:
im=interact(plot_pearson_aes)

interactive(children=(Dropdown(description='state_name', options=('SubByte9', 'SubByte8', 'SubByte4', 'SubByte…